In [1]:
import os
import re

import django
import fs.path
import numpy as np
import pandas as pd
from django.core.exceptions import ObjectDoesNotExist
from fs.osfs import OSFS
from more_itertools import sample

if not os.getcwd().endswith('multidex'):
    os.chdir('..')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "multidex.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

from plotter.models import ZSpec

In [ ]:
for spec in ZSpec.objects.all():
    spec.delete()

In [2]:
from ingest.cli import find_ingest_files
from pathlib import Path

root = Path("/home/michael/Desktop/asdf_output-20210518T223003Z-001")
# output_image_dir = output_fs.getsyspath("static_in_pro/our_static/img")

In [3]:
from ingest.cli import *
from ingest.thumbpipe import default_thumbnailer

In [4]:
marslab_files, context_files = find_ingest_files(root, recursive=True)
context_df = process_context_files(context_files, default_thumbnailer())

making thumbnails


In [5]:
# marslab_files = tuple(marslab_files)
for marslab_file in marslab_files:
    obs_images = str(match_obs_images(marslab_file, context_df))
    if obs_images != "{}":
        pass
#         print("found matching images: " + obs_images)
    else:
        print("no matching images found for " + marslab_file)

no matching images found for /home/michael/Desktop/asdf_output-20210518T223003Z-001/asdf_output/marslab-csvs/SITE3_DRIVE2208_RMS518_SOL69_SEQIDzcam03120_ZOOM110-cliff2-marslab.csv
no matching images found for /home/michael/Desktop/asdf_output-20210518T223003Z-001/asdf_output/marslab-csvs/SITE3_DRIVE2208_RMS518_SOL69_SEQIDzcam03120_ZOOM110-foreground1-marslab.csv
no matching images found for /home/michael/Desktop/asdf_output-20210518T223003Z-001/asdf_output/marslab-csvs/SITE3_DRIVE2208_RMS522_SOL69_SEQIDzcam03120_ZOOM110-rim-marslab.csv
no matching images found for /home/michael/Desktop/asdf_output-20210518T223003Z-001/asdf_output/marslab-csvs/SITE3_DRIVE1266_RMS44_SOL33_SEQIDzcam03106_ZOOM110-rocks-marslab.csv
no matching images found for /home/michael/Desktop/asdf_output-20210518T223003Z-001/asdf_output/marslab-csvs/SITE3_DRIVE2208_RMS522_SOL69_SEQIDzcam03120_ZOOM110-mid-marslab.csv
no matching images found for /home/michael/Desktop/asdf_output-20210518T223003Z-001/asdf_output/marslab

In [ ]:
# marslab_files = tuple(marslab_files)
for marslab_file in marslab_files:
    print("ingesting spectra from " + marslab_file)
    obs_images = str(match_obs_images(marslab_file, context_df))
    if obs_images != "{}":
        print("found matching images: " + obs_images)
    else:
        print("no matching images found")
    frame = pd.read_csv(marslab_file)
    y_to_bool(frame, ZCAM_BOOL_FIELDS)
    frame = frame.replace(["-", "", " "], np.nan)
    frame.columns = [col.lower() for col in frame.columns]
    frame = frame[[col for col in frame.columns if col not in USELESS_COLUMNS]]
    colors = []
    for _, row in frame.iterrows():
        row = row.dropna()
        # if there are missing filters anywhere in the column, including for other
        # spectra, pandas will read the column as object / string, which will 
        # cause confusion when we compute averaged filters
        for filt in set(ZSpec.filters).intersection(row.index):
            row[filt] = float(row[filt])
        metadata = dict(row) | {
            'filename': fs.path.split(marslab_file)[-1], 'images': obs_images
        }
        try:
            spectrum = ZSpec(**metadata)
            spectrum.clean()
            spectrum.save()
            colors.append(row['color'] + " " + row.get('feature'))
        except KeyboardInterrupt:
            raise
        except Exception:
            print("couldn't ingest " + row[color] + ": " + str(Exception))
    print("successfully ingested " + ", ".join(colors))


In [ ]:
obs_images

In [ ]:
obs_images

In [ ]:
file_stem = fs.path.split(marslab_file)[-1].replace("-marslab.csv", "")

In [ ]:
file_stem

In [ ]:
from plotter_utils import model_metadata_df

In [ ]:
ZSpec.objects.all()

In [ ]:
len(ZSpec.objects.all())

In [ ]:
ZSpec._meta.fields

In [ ]:
obs_images

In [ ]:
obs_images

In [ ]:
images = {
    'left': 
}

In [ ]:
pd.DataFrame.to_dict?

In [ ]:
context_df

In [ ]:
conte

In [ ]:
frame.columns = [column.lower() for column in frame.columns]

In [ ]:
spec_files = [
    file for file in input_fs.listdir('') if (
        file.endswith('-marslab.csv') and file.startswith('Sol'))
]
for spec_file in spec_files:
    frame = pd.read_csv(input_fs.getsyspath(spec_file))
    frame.columns = [column.lower() for column in frame.columns]
    observation = metaframe.loc[metaframe['seq_id'] == frame['seq_id'][0]]
    # hacky hacky -- but temporary
    if len(observation) > 1: 
        pointing = re.search(r"(?<=Pointing)\d", spec_file).group()
        observation = observation.loc[observation['name'].str.contains(str(pointing))]
    assert len(observation == 1)
    for _, row in frame.iterrows():
        if row['float'] == 'Y':
            row['float'] = True
        else:
            row['float'] = False
        row = row.replace(['-','',' '], np.nan).dropna()
        # if there are missing filters anywhere in the column, including for other
        # spectra, pandas will read the column
        # as object / string, which will cause confusion when we
        # compute averaged filters
        for filt in ZSpec.filters:
            if filt in row.index:
                row[filt] = float(row[filt])
        # because we're manually associating
        row.drop(['sol', 'seq_id', 'instrument'], inplace=True)
        metadata = dict(row) | observation.iloc[0].dropna().to_dict() | {
            'filename': spec_file
        }
        spectrum = ZSpec(**metadata)
        spectrum.clean()
        spectrum.save()

In [ ]:
path = 

input_fs = OSFS("/home/michael/Desktop/mcam_spect_data_conversion/data/zcam")
output_fs = OSFS('.')


In [ ]:
for spec in ZSpec.objects.all():
    spec.delete()

In [ ]:
input_fs = OSFS("/home/michael/Desktop/mcam_spect_data_conversion/data/zcam")
output_fs = OSFS('.')
output_image_dir = output_fs.getsyspath("static_in_pro/our_static/img")

In [ ]:
def reg_seq_id(seq_id_str):
    seq_id_str = seq_id_str.replace(" ", "")
    return 'zcam' + format(int(seq_id_str.split('zcam')[1]), "0>5d")

metaframe = pd.read_csv(input_fs.getsyspath('Metadata-marslab.csv'))
metaframe.columns = [column.lower() for column in metaframe.columns]
# add NaNs back in so we can programmatically delete them
metaframe.replace('-', np.nan, inplace=True)
# we're turning these to ints when we ingest them,
# but python doesn't like statements like int('3.0'),
# so turn to float as an intermediate step
for column in [
    'sol', 'site', 'drive', 'rover_elevation', 'target_elevation', 'tau',
    'focal_distance', 'incidence_angle', 'emission_angle',
    'phase_angle', 'l_s', 'site', 'drive', 'lat', 'lon', 'odometry',
]:
    metaframe[column] = metaframe[column].astype('float')
metaframe['ltst'] = metaframe['ltst'].astype('datetime64')
metaframe['seq_id'] = metaframe['seq_id'].map(reg_seq_id)
metaframe.loc[
    metaframe['sclk'].notna(), 'sclk'
] = metaframe.loc[metaframe['sclk'].notna(), 'sclk'].astype(float).astype(int)

In [ ]:
temp_images = [
    image for image in input_fs.listdir('temp_images')
]

In [ ]:
spec_files = [
    file for file in input_fs.listdir('') if (
        file.endswith('-marslab.csv') and file.startswith('Sol'))
]
for spec_file in spec_files:
    frame = pd.read_csv(input_fs.getsyspath(spec_file))
    frame.columns = [column.lower() for column in frame.columns]
    observation = metaframe.loc[metaframe['seq_id'] == frame['seq_id'][0]]
    # hacky hacky -- but temporary
    if len(observation) > 1: 
        pointing = re.search(r"(?<=Pointing)\d", spec_file).group()
        observation = observation.loc[observation['name'].str.contains(str(pointing))]
    assert len(observation == 1)
    for _, row in frame.iterrows():
        if row['float'] == 'Y':
            row['float'] = True
        else:
            row['float'] = False
        row = row.replace(['-','',' '], np.nan).dropna()
        # if there are missing filters anywhere in the column, including for other
        # spectra, pandas will read the column
        # as object / string, which will cause confusion when we
        # compute averaged filters
        for filt in ZSpec.filters:
            if filt in row.index:
                row[filt] = float(row[filt])
        # because we're manually associating
        row.drop(['sol', 'seq_id', 'instrument'], inplace=True)
        metadata = dict(row) | observation.iloc[0].dropna().to_dict() | {
            'filename': spec_file
        }
        spectrum = ZSpec(**metadata)
        spectrum.clean()
        spectrum.save()

In [ ]:
len(ZSpec.objects.all())

In [ ]:
output_image_dir

In [ ]:
z

In [ ]:
spec.seq_id

In [ ]:
os.listdir(output_image_dir)

In [ ]:
spec.filename

In [ ]:
import re

In [ ]:
for spec in ZSpec.objects.all():
    images = [
        image for image in os.listdir(output_image_dir + '/roi_browse')
        if spec.seq_id in image
    ]
    if 'pointing' in spec.filename.lower():
        pointing = re.search(r"(?<=Pointing)\d", spec_file).group()
        images = [image for image in images if 'pointing' + pointing in image]
    image_dict = {}
    for image in images:
        if 'rgb' in image:
            image_dict['rgb'] = image
        if 'enhanced' in image:
            image_dict['enhanced'] = image
    spec.images = str(image_dict)
    spec.clean()
    spec.save()

In [ ]:
from random import choice

In [ ]:
spec = choice(ZSpec.objects.all())

In [ ]:
from plotter.spectrum_ops import *

In [ ]:
unc = """L0R:3.3
L0G:3.3
L0B:3.7
L1:1.4
L2:1.1
L3:0.2
L4:1.8
L5:1.6
L6:0.4
R0R:3.7
R0G:4.1
R0B:4.6
R1:0.4
R2:0.3
R3:0.6
R4:0.5
R5:0.8
R6:0.4""".split("\n")

In [ ]:
unc

In [ ]:
{
    pair.split(":")[0]: float(pair.split(":")[1])
    for pair in unc
}

In [ ]:
filter_df = filter_df_from_queryset(
    ZSpec.objects.all(),
    average_filters=False,
    scale_to=None,
    r_star=True,
    )

In [ ]:
from marslab.compatibility import INSTRUMENT_UNCERTAINTIES
from itertools import chain, product

In [ ]:
spec_model = ZSpec
instrument = spec_model.instrument
unc = INSTRUMENT_UNCERTAINTIES[instrument]
function = ratio
# filters = ['L5', 'R3', 'L4']
filters = ['L5', 'R3']

In [ ]:
def compute_minmax_spec_error(filter_df, spec_model, spec_op, *filters):
    # cartesian product of these sets gives all possible sign combos for error high, error low,
    # i.e., crude bounds for the hull of the range of possible measurements
    unc = INSTRUMENT_UNCERTAINTIES[spec_model.instrument]
    corners = product(*[[1, -1] for filt in filters])
    bounds_df_list = []
    # apply these signs to uncertainty values, getting a list of dataframes
    # giving values of all measurements in set at the upper / lower bound combinations
    # for uncertainties associated with each relevant filter.
    for corner in corners:
        corner_series_list = []
        for filt_ix, sign in enumerate(corner):
            filt = filters[filt_ix]
            corner_series_list.append(
                filter_df[filt]  + filter_df[filt] * corner[filt_ix] * unc[filt] / 100
            )
        corner_df = pd.concat(corner_series_list, axis=1)
        # record the value of the spectrum op for each of these bounding dataframes.
        bounds_df_list.append(function(corner_df, spec_model, *filters)[0])   
    # compute the nominal value and compare it to values at these bounds
    possible_values = pd.concat(bounds_df_list, axis=1)
    nominal_value = function(filter_df, spec_model, *filters)[0]
    offsets = possible_values.sub(nominal_value, axis=0) 
    # then min / max of each of these gives us an error estimate for each spectrum
    return offsets.min(axis=1), offsets.max(axis=1)

In [ ]:
# cartesian product of these sets gives all possible sign combos for error high, error low,
# i.e., crude bounds for the hull of the range of possible measurements
corners = product(*[[1, -1] for filt in filters])
bounds_df_list = []
# apply these signs to uncertainty values, getting a list of dataframes
# giving values of all measurements in set at the upper / lower bound combinations
# for uncertainties associated with each relevant filter.
for corner in corners:
    corner_series_list = []
    for filt_ix, sign in enumerate(corner):
        filt = filters[filt_ix]
        corner_series_list.append(
            filter_df[filt]  + filter_df[filt] * corner[filt_ix] * unc[filt] / 100
        )
    corner_df = pd.concat(corner_series_list, axis=1)
    # record the value of the spectrum op for each of these bounding dataframes.
    bounds_df_list.append(function(corner_df, spec_model, *filters)[0])    

In [ ]:
# compute the nominal value and compare it to values at these bounds
possible_values = pd.concat(bounds_df_list, axis=1)
nominal_value = function(filter_df, spec_model, *filters)[0]
offsets = possible_values.sub(nominal_value, axis=0) 
# then min / max of each of these gives us an error estimate for each spectrum
error_low = offsets.min(axis=1)
error_high = offsets.max(axis=1)

In [ ]:
nominal_value = function(filter_df, spec_model, *filters)[0]

In [ ]:
possible_values

In [ ]:
nominal_value

In [ ]:
nominal_value = function(filter_df, spec_model, *filters)[0]
bound = function(bounds_df_list[0], spec_model, *filters)[0] 
bound - nominal_value

In [ ]:
# note that band_depth_min is a special case

nominal_value = band_avg(filter_df, ZSpec, 'L1', 'R1')[0]
band_avg(bounds_df_list[0], ZSpec, 'L1', 'R1')[0] - nominal_value

In [ ]:
corner_df - nominal_value

In [ ]:
list(product(filter_df_low.columns, filter_df_high))

In [ ]:
filter_df_low